**Video Processing & Video Object Tracking**

In [ ]:
!wget -O video.mp4 https://github.com/heshameraqi/Image_Processing_Computer_Vision_1/raw/main/tracking_slow_traffic_small.mp4

import numpy as np
import cv2
import matplotlib.pyplot as plt

# Capture video and take first frame of the video
cap = cv2.VideoCapture('video.mp4')
ret, frame = cap.read()
plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

# Initial location of window for tracked object
x, y, width, height = 300, 200, 100, 50 # White Car
track_window = (x, y ,width, height)

# Set up the object (ROI) for tracking
obj = frame[y:y+height, x : x+width]
hsv_obj = cv2.cvtColor(obj, cv2.COLOR_BGR2HSV)
mask = cv2.inRange(hsv_obj, (0., 60., 32.), (180., 255., 255))
obj_hist = cv2.calcHist([hsv_obj], [0], mask, [180], [0, 180])
plt.show()
plt.imshow(cv2.cvtColor(obj, cv2.COLOR_BGR2RGB))

# Mean Shift algorithm termination criteria, either 10 iteration reached or move by at least 1 pt
term_crit = ( cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1 )
i = 0
while(True):
    ret, frame = cap.read()
    if ret == True:
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV) # New frame
        # Back Projection: find a feature in the new frame image by seaching for its histogram model, returns probability image
        dst = cv2.calcBackProject([hsv],[0], obj_hist, [0,180], 1)

        # Apply meanshift to get the new location
        ret, track_window = cv2.meanShift(dst, track_window, term_crit)

        # Draw tracking result on image
        x,y,w,h = track_window
        result_image = cv2.rectangle(frame, (x,y), (x+w, y+h), 255, 3)
        cv2.imwrite("Result%04d.jpg" % i, result_image)
        i+=1
    else:
        break

cap.release() # Release video reader

# Create the resulted video file, then download it manually if needed and watch the result
!ffmpeg -framerate 30 -r 30 -i 'Result%04d.jpg' -vcodec mpeg4 -y result.mp4 # Save video
from google.colab import files
files.download('result.mp4')

**Exercise: Track the woman with the bicycle**